In [ ]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

from datetime import datetime
import re
from collections import defaultdict
import time
import json

ua = UserAgent()
fakeua = ua.random

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_Android_smartphones"

# GET request from url and parse via BeautifulSoup
resp = requests.get(url)
#resp.encoding = 'utf-8' # encoded with format utf-8 for chinese character
soup = BeautifulSoup(resp.text, "html.parser")

In [ ]:
tables = soup.find_all("table", class_="wikitable")
rows = []
for i in tables:
    rows.extend(i.select("tbody tr th"))

In [ ]:
phones = [th.text.partition("(")[0].strip() for th in th_rows]
phones[5]

In [ ]:
brands = set()
for p in phones:
    brands.add(p.split(" ")[0])
brands = list(brands)
brands = sorted(brands)
for b in range(len(brands)):
    if brands[b] == "Asus":
        brands = brands[b:]
        break

In [ ]:
phone_dict = defaultdict(list)
for b in brands:
    if b == "Samsung":
        for p in phones:
            if p.split(" ")[0] == b:
                phone = p.replace(b + " Galaxy", "").replace("5G", "").strip()
                if len(phone) > 0:
                    phone_dict[b].append(phone)
    elif b == "POCO":
        for p in phones:
            if p.split(" ")[0] == b:
                phone = p.replace("5G", "").strip()
                if len(phone) > 0:
                    phone_dict["Xiaomi"].append(phone)
    else:
        for p in phones:
            if p.split(" ")[0] == b:
                phone = p.replace(b, "").replace("5G", "").strip()
                if len(phone) > 0:
                    phone_dict[b].append(phone)

# phone_dict

## Android

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_Android_smartphones"
resp = requests.get(url)
soup = BeautifulSoup(resp.text, "html.parser")

tables = soup.find_all("table", class_="wikitable")
rows = []
for i in tables:
    rows.extend(i.select("tbody tr th"))

phones = [a.text.partition("(")[0].strip() for a in rows]
phones = [ p for p in phones if len(p.split()) > 1]

brands = set()
for p in phones:
    brands.add(p.split(" ")[0])
brands = list(brands)
brands = sorted(brands)
for b in range(len(brands)):
    if brands[b] == "Asus":
        brands = brands[b:]
        break

phone_dict = defaultdict(list)
for b in brands:
    if b == "Samsung":
        for p in phones:
            if p.split(" ")[0] == b:
                if "/" in p:
                    p_lst = p.split("/")
                    for series in p_lst[1:]:
                        sub_p = p_lst[0].replace(b + " Galaxy", "").replace("5G", "").strip()
                        if series == "+":
                            phone = sub_p + series
                        else:
                            phone = sub_p + " " + series
                        if len(phone) > 0:
                            phone_dict[b].append(phone)
                    phone_dict[b].append(sub_p)
                else:
                    phone = p.replace(b + " Galaxy", "").replace("5G", "").strip()
                    if len(phone) > 0:
                        phone_dict[b].append(phone)
    elif b == "POCO":
        for p in phones:
            if p.split(" ")[0] == b:
                phone = p.replace("5G", "").strip()
                if len(phone) > 0:
                    phone_dict["Xiaomi"].append(phone)
    elif b == "Pixel":
        for p in phones:
            if p.split(" ")[0] == b:
                if "/" in p:
                    p_lst = p.split("/")
                    for series in p_lst[1:]:
                        sub_p = p_lst[0].replace("5G", "").strip()
                        if series == "+":
                            phone = sub_p + series
                        else:
                            phone = sub_p + " " + series
                        if len(phone) > 0:
                            phone_dict[b].append(phone)
                    phone_dict[b].append(sub_p)
                else:
                    phone = p.replace("5G", "").strip()
                    if len(phone) > 0:
                        phone_dict[b].append(phone)
    else:
        for p in phones:
            if p.split(" ")[0] == b:
                if "/" in p:
                    p_lst = p.split("/")
                    for series in p_lst[1:]:
                        sub_p = p_lst[0].replace(b, "").replace("5G", "").strip()
                        if series == "+":
                            phone = sub_p + series
                        else:
                            phone = sub_p + " " + series
                        if len(phone) > 0:
                            phone_dict[b].append(phone)
                    phone_dict[b].append(sub_p)
                else:
                    phone = p.replace(b, "").replace("5G", "").strip()
                    if len(phone) > 0:
                        phone_dict[b].append(phone)
                    phone_dict[b].append(phone)
phone_dict["Asus"].append("ROG Phone 2")
phone_dict.pop("Galaxy")
for k in phone_dict.keys():
    phone_dict[k] = sorted(phone_dict[k], key=lambda x: len(x), reverse=True)

In [ ]:
import pprint
print(len(phone_dict))
pprint.pprint(phone_dict.keys())
pprint.pprint(phone_dict["Redmi"])

## iPhone

In [ ]:
url = "https://www.theiphonewiki.com/wiki/List_of_iPhones"
resp = requests.get(url)
soup = BeautifulSoup(resp.text, "html.parser")

table = soup.find("div", id="toc")
iphones = []
for span in table.select("ul li span"):
    if "iPhone" in str(span):
        if ("SE" in str(span)) and ("1" in str(span)):
            iphones.append("iPhone SE")
        elif ("SE" in str(span)) and ("2" in str(span)):
            iphones.append("iPhone SE2")
        else:
            iphones.append(span.text)
iphones = sorted(iphones, key= lambda x:len(x), reverse=True)
phone_dict["iPhone"] = iphones
iphones

In [ ]:
phone_brand_set = set()
for k in phone_dict.keys():
    for v in phone_dict[k]:
        for iphone in [ i.replace("iPhone", "").strip() for i in phone_dict["iPhone"] ]:
            if v == iphone:
                phone_brand_set.add(v)
iphone_matches = sorted([v for v in list(set([ i.replace("iPhone", "").strip() for i in phone_dict["iPhone"] ]) - phone_brand_set) if len(v) > 0], key=lambda x: len(x), reverse=True)
iphone_matches = [ i for i in iphone_matches if ("11" in i) or ("12" in i) or ("SE2" in i)]
iphone_matches.remove("12")
iphone_matches.remove("11")
iphone_matches

## Export phone_dict

In [ ]:
import pprint
print(len(phone_dict))
pprint.pprint(phone_dict)

In [ ]:
with open('phone_catalog.json', 'w') as fp:
    json.dump(phone_dict, fp, sort_keys=True, indent=4)

In [ ]:
# Read
with open('phone_catalog.json', 'r') as fp:
    phone_dict = json.load(fp)

# crawl PTT

In [ ]:
def get_last_page(): # 20596
    ua = UserAgent()
    fakeua = ua.random
    headers = {"Origin": "https://www.ptt.cc",
            "Referer": "https://fonts.googleapis.com/",
            "sec-ch-ua-mobile": "?0",
            "User-Agent": fakeua}
    url = "https://www.ptt.cc/bbs/mobilesales/index.html"
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text, "html.parser")
    previous = soup.find_all("a", class_="btn wide")[1]["href"]
    previous_page_num = int(re.findall(r"\d+", previous)[0])
    return previous_page_num
last_page = get_last_page()
last_page

In [ ]:
from crawler_model import Ptt
max_page = Ptt().select_max_page_number("mobilesales")
max_page

In [ ]:
def crawl_mobilesales(links, phone_dict, sold = False):
    db = pymysql.connect(host=HOST, user=USER, password=PWD, database="smartphone")
    for link in links:
        url = link
        # url = "https://www.ptt.cc" + link
        # print(url)
        # GET request from url and parse via BeautifulSoup
        ua = UserAgent()
        fakeua = ua.random
        headers = {"Origin": "https://www.ptt.cc",
                "Referer": "https://fonts.googleapis.com/",
                "sec-ch-ua-mobile": "?0",
                "User-Agent": fakeua}
        resp = requests.get(url, headers=headers)
        if "200" in str(resp):
            try:
                soup = BeautifulSoup(resp.text, "html.parser")

                # sold
                body = soup.find("body")
                if "已售" in body or "售出" in body:
                    sold = True
                else:
                    sold = False
                # accout, source, title, created_at
                info = [ i.text.strip() for i in soup.findAll("span", class_="article-meta-value")]
                try:
                    title = re.findall(r'售物品.+\n', str(soup.find("div", id="main-content")))[0][4:].strip()      
                    
                    if len(title) == 0:
                        title = info[2]     

                except IndexError:
                    title = info[2]

                if ("小米" in title.lower()) or ("redmi" in title.lower()) or ("poco" in title.lower()):
                    title = "Xiaomi" + " " + title
                if ("紅米" in title.lower()):
                    title = "Redmi" + " " + title
                if ("三星" in title.lower()):
                    title = "Samsung" + " " + title

                title = " ".join(re.findall(r"[a-zA-Z0-9+]+", title))
                storage = None
                if "rog" in title.lower():
                    for phone in phone_dict["Asus"]:
                        if phone.lower() in title.lower():
                            for s in  ["32", "64", "128", "256", "512"]:
                                if s in title:
                                    storage = int(s)
                                    break
                            title = "Asus" + " " + phone
                            break

                samsung_matches = ["note20ultra", "s20ultra", "s21ultra"]
                if any(x in title.lower().replace(" ", "") for x in samsung_matches):
                    for phone in phone_dict["Samsung"]:
                        if phone.lower().replace(" ", "") in title.lower().replace(" ", ""):
                            for s in  ["32", "64", "128", "256", "512"]:
                                if s in title:
                                        storage = int(s)
                                        break
                            title = "Samsung Galaxy" + " " + phone
                            break

                for k in phone_dict.keys():
                    if k.lower() in title.lower().replace(" ", ""): 
                        for phone in phone_dict[k]:
                            if phone.lower().replace(" ", "") in title.lower().replace(" ", ""):
                                for s in  ["32", "64", "128", "256", "512"]:
                                    if s in title:
                                        storage = int(s)
                                        break
                                if k == "Samsung":
                                    title = k + " " + "Galaxy" + " " + phone
                                elif k == "iPhone" or k == "Pixel":
                                    title = phone
                                else:
                                    title = k + " " + phone
                                break
                        break

                if title.split(" ")[0] in phone_dict.keys():
                    pass
                else:
                    for x in iphone_matches:
                        if x.lower().replace(" ", "") in title.lower().replace(" ", ""):
                            print(x)
                            for phone in phone_dict["iPhone"]:
                                if x == phone.lower().replace(" ", ""):
                                    for s in  ["32", "64", "128", "256", "512"]:
                                        if s in title:
                                            storage = int(s)
                                            break
                                    title = phone
                                    print("iPhone:", title)
                                    break
                            break

                try:
                    account = info[0]
                    created_at = datetime.strptime(info[3][3:].strip(), '%b %d %H:%M:%S %Y')
                except IndexError:
                    account = None
                    created_at = " ".join(re.findall(r'時間.+\n', str(soup.find("div", id="main-content")))[0].split(" ")[2:]).strip()
                    created_at = datetime.strptime(created_at, '%b %d %H:%M:%S %Y')


                # status
                try:

                    if ("全新" in info[2]):
                        new = True
                    elif ("近全新" in info[2]) or ("外觀全新" in info[2]) or ("全新耳機" in info[2]):
                        new = False
                    else:
                        new = False
                except IndexError:
                    new = False

                # price
                try:
                    wording = "價格"
                    price = re.findall(r'售價格.+\n', str(soup.find("div", id="main-content")))
                    if len(price) == 0:
                        wording = "金額"
                        price = re.findall(r'售金額.+\n', str(soup.find("div", id="main-content")))
                    price = price[0].strip().replace(",", "")
                    price = int(re.search(r'\d+', price).group())
                except IndexError:
                    if wording == "金額":
                        price = re.findall(r'售金額.+', str(soup.find("div", id="main-content")).replace("\n", "").replace(",", "").replace(" ", ""))
                        price = int(re.findall(r'\d+', price)[0])
                    else:
                        price = re.findall(r'售價格.+', str(soup.find("div", id="main-content")).replace("\n", "").replace(",", "").replace(" ", ""))[0].strip()
                        price = int(re.findall(r'\d+', price)[0])
                except AttributeError:
                    sold = True
                    price = None
                
                # items included
                try:
                    box = re.findall(r'盒裝.+\n', str(soup.find("div", id="main-content")))[0].strip()
                    if len(box) <= 5:
                        box = re.findall(r'盒裝內含.+購買', str(soup.find("div", id="main-content")).replace("\n", ""))[0]
                        box = box.replace(" ", "")[:-2]
                except IndexError:
                    box = None

                data.append([title, storage, price, new, sold, account, box, url, created_at, page])
                raw_data.append([url, str(soup.find("body"))])
                db.ping()
                cursor = db.cursor(pymysql.cursors.DictCursor)  
                cursor.execute('''
                    INSERT INTO ptt
                    (`title`, `storage`, `price`, `new`, `sold`, 
                        `account`, `box`, `link`, `created_at`, 
                        `source`)
                    VALUES (%s, %s, %s, %s, %s,
                            %s, %s, %s, %s, %s,
                            %s)
                    ON DUPLICATE KEY UPDATE `title` = %s, 
                                            `storage` = %s, 
                                            `price` = %s, 
                                            `new` = %s, 
                                            `sold` = %s, 
                                            `account` = %s, 
                                            `box` = %s, 
                                            `created_at` = %s, 
                                            `source` = %s;
                    ''', (title, storage, price, new, sold, account, box, url, created_at, "mobilesales",
                        title, storage, price, new, sold, account, box, created_at, "mobilesales"))
                db.commit()
                print(url)
            except:
                error_links.append(url)

In [ ]:
from crawler_model import Ptt

In [ ]:
raw_data = []
data = []
error_links = []
last_page = get_last_page()
init = time.time()
for page in range(20699, 20000, -1):
    start = time.time()
    ua = UserAgent()
    fakeua = ua.random
    headers = {"Origin": "https://www.ptt.cc",
            "Referer": "https://fonts.googleapis.com/",
            "sec-ch-ua-mobile": "?0",
            "User-Agent": fakeua}

    url = "https://www.ptt.cc/bbs/mobilesales/index{}.html".format(page)
    print(url)
    # GET request from url and parse via BeautifulSoup
    resp = requests.get(url, headers=headers)
    #resp.encoding = 'utf-8' # encoded with format utf-8 for chinese character
    index_soup = BeautifulSoup(resp.text, "html.parser")
    title = index_soup.findAll("div", class_="title")
    links = [ t.find("a")["href"] for t in title if ("賣" in str(t)) and ("已售" not in str(t)) and ("售出" not in str(t))]
    sold_links = [ t.find("a")["href"] for t in title if ("已售" in str(t)) or ("售出" in str(t))]

    crawl_mobilesales(links, page, sold=False, phone_dict=phone_dict)
    crawl_mobilesales(sold_links, page, sold=True, phone_dict=phone_dict)
    print(time.time()-start, time.time()-init)
    if page%100 == 0:
        db = Ptt()
        db.insert_post(raw_data, data)
        raw_data = []
        data = []
db = Ptt()
db.insert_post(raw_data, data)
raw_data = []
data = []        

In [ ]:
print(len(error_links))

In [ ]:
import pprint
print(len(data))
pprint.pprint(data[-100:])

# Test

In [ ]:
ua = UserAgent()
fakeua = ua.random
headers = {"Origin": "https://www.ptt.cc",
        "Referer": "https://fonts.googleapis.com/",
        "sec-ch-ua-mobile": "?0",
        "User-Agent": fakeua}

def test_one_url(url):
    phone_brand_set = set()
    for k in phone_dict.keys():
        for v in phone_dict[k]:
            for iphone in [ i.replace("iPhone", "").strip() for i in phone_dict["iPhone"] ]:
                if v == iphone:
                    phone_brand_set.add(v)
    iphone_matches = sorted([v for v in list(set([ i.replace("iPhone", "").strip() for i in phone_dict["iPhone"] ]) - phone_brand_set) if len(v) > 0], key=lambda x: len(x), reverse=True)
    iphone_matches.remove("12")
    iphone_matches.remove("11")
        
    ua = UserAgent()
    fakeua = ua.random
    headers = {"Origin": "https://www.ptt.cc",
            "Referer": "https://fonts.googleapis.com/",
            "sec-ch-ua-mobile": "?0",
            "User-Agent": fakeua}
    resp = requests.get(url, headers=headers)
    if "200" in str(resp):
        soup = BeautifulSoup(resp.text, "html.parser")

        # basic info: accout, board, title, created_at
        info = [ i.text.strip() for i in soup.findAll("span", class_="article-meta-value")]
        try:
            title = re.findall(r'售物品.+\n', str(soup.find("div", id="main-content")))[0][4:].strip()      
            
            if len(title) == 0:
                title = info[2]     

        except IndexError:
            title = info[2]

        if ("小米" in title.lower()) or ("redmi" in title.lower()) or ("poco" in title.lower()):
            title = "Xiaomi" + " " + title
        if ("紅米" in title.lower()):
            title = "Redmi" + " " + title
        if ("三星" in title.lower()):
            title = "Samsung" + " " + title

        title = " ".join(re.findall(r"[a-zA-Z0-9+]+", title))
        storage = None
        if "rog" in title.lower():
            for phone in phone_dict["Asus"]:
                if phone.lower() in title.lower():
                    for s in  ["32", "64", "128", "256", "512"]:
                        if s in title:
                            storage = int(s)
                            break
                    title = "Asus" + " " + phone
                    break

        samsung_matches = ["note20ultra", "s20ultra", "s21ultra"]
        if any(x in title.lower().replace(" ", "") for x in samsung_matches):
            for phone in phone_dict["Samsung"]:
                if phone.lower().replace(" ", "") in title.lower().replace(" ", ""):
                    for s in  ["32", "64", "128", "256", "512"]:
                        if s in title:
                                storage = int(s)
                                break
                    title = "Samsung Galaxy" + " " + phone
                    break

        for k in phone_dict.keys():
            if k.lower() in title.lower().replace(" ", ""): 
                print(k)
                for phone in phone_dict[k]:
                    if phone.lower().replace(" ", "") in title.lower().replace(" ", ""):
                        print(phone)
                        for s in  ["32", "64", "128", "256", "512"]:
                            if s in title:
                                storage = int(s)
                                break
                        if k == "Samsung":
                            title = k + " " + "Galaxy" + " " + phone
                        elif k == "iPhone":
                            title = phone
                        else:
                            title = k + " " + phone
                        break
                break

        if title.split(" ")[0] in phone_dict.keys():
            pass
        else:
            for x in iphone_matches:
                if x.lower().replace(" ", "") in title.lower().replace(" ", ""):
                    print(x)
                    for phone in phone_dict["iPhone"]:
                        if x == phone.lower().replace(" ", ""):
                            for s in  ["32", "64", "128", "256", "512"]:
                                if s in title:
                                    storage = int(s)
                                    break
                            title = phone
                            print("iPhone:", title)
                            break
                    break

        try:
            account = info[0]
            created_at = datetime.strptime(info[3][3:].strip(), '%b %d %H:%M:%S %Y')
        except IndexError:
            account = None
            created_at = " ".join(re.findall(r'時間.+\n', str(soup.find("div", id="main-content")))[0].split(" ")[2:]).strip()
            created_at = datetime.strptime(created_at, '%b %d %H:%M:%S %Y')


        # status
        try:
            # status = re.findall(r'物品狀況.+\n', str(soup.find("div", id="main-content")))[0].strip()
            # if len(status) <= 5:
            #     status = re.findall(r'狀況.+盒裝內含', str(soup.find("div", id="main-content")).replace("\n", ""))[0]

            if ("全新" in info[2]):
                new = True
            elif ("近全新" in info[2]) or ("外觀全新" in info[2]) or ("全新耳機" in info[2]):
                new = False
            else:
                new = False
        except IndexError:
            new = False

        # price
        try:
            wording = "價格"
            price = re.findall(r'售價格.+\n', str(soup.find("div", id="main-content")))
            if len(price) == 0:
                wording = "金額"
                price = re.findall(r'售金額.+\n', str(soup.find("div", id="main-content")))
            price = price[0].strip().replace(",", "")
            price = int(re.search(r'\d+', price).group())
        except IndexError:
            if wording == "金額":
                price = re.findall(r'售金額.+', str(soup.find("div", id="main-content")).replace("\n", "").replace(",", "").replace(" ", ""))
                price = int(re.findall(r'\d+', price)[0])
            else:
                price = re.findall(r'售價格.+', str(soup.find("div", id="main-content")).replace("\n", "").replace(",", "").replace(" ", ""))[0].strip()
                price = int(re.findall(r'\d+', price)[0])
        except AttributeError:
            sold = True
            price = None
        
        # items included
        try:
            box = re.findall(r'盒裝.+\n', str(soup.find("div", id="main-content")))[0].strip()
            if len(box) <= 5:
                box = re.findall(r'盒裝內含.+購買', str(soup.find("div", id="main-content")).replace("\n", ""))[0]
                box = box.replace(" ", "")[:-2]
        except IndexError:
            box = None

    print([title, storage, price, new, account, box, url, created_at])

In [ ]:
test_one_url("https://www.ptt.cc/bbs/mobilesales/M.1588315298.A.599.html")

# Insert into db

In [ ]:
print(data[0])

In [ ]:
db = Ptt()
db.insert_post(raw_data, data)

# iPhone 

In [ ]:
phone_brand_list = []
for k in phone_dict.keys():
    for v in phone_dict[k]:
        for iphone in [ i.replace("iPhone", "").strip() for i in phone_dict["iPhone"] ]:
            if v == iphone:
                phone_brand_list.append((v, k))
phone_brand_list = sorted(phone_brand_list)
pprint.pprint(phone_brand_list)

In [ ]:
phone_dict["iPhone"]

In [ ]:
phone_brand_set = set()
for k in phone_dict.keys():
    for v in phone_dict[k]:
        for iphone in [ i.replace("iPhone", "").strip() for i in phone_dict["iPhone"] ]:
            if v == iphone:
                phone_brand_set.add(v)
iphone_matches = sorted([v for v in list(set([ i.replace("iPhone", "").strip() for i in phone_dict["iPhone"] ]) - phone_brand_set) if len(v) > 0], reverse=True)
iphone_matches

In [ ]:
iphone_matches = sorted([v for v in list(set([ i.replace("iPhone", "").strip() for i in phone_dict["iPhone"] ]) - phone_brand_set) if len(v) > 0], reverse=True)
iphone_matches

# 回測

In [ ]:
def crawl_mobilesales(links, phone_dict, sold = False):
    db = pymysql.connect(host=HOST, user=USER, password=PWD, database="smartphone")
    for link in links:
        url = link
        ua = UserAgent()
        fakeua = ua.random
        headers = {"Origin": "https://www.ptt.cc",
                "Referer": "https://fonts.googleapis.com/",
                "sec-ch-ua-mobile": "?0",
                "User-Agent": fakeua}

        resp = requests.get(url, headers=headers)
        if "200" in str(resp):
            soup = BeautifulSoup(resp.text, "html.parser")

            # sold
            body = soup.find("body")
            if "已售" in body or "售出" in body:
                sold = True
            else:
                sold = False
            # accout, source, title, created_at
            info = [ i.text.strip() for i in soup.findAll("span", class_="article-meta-value")]
            try:
                title = re.findall(r'售物品.+\n', str(soup.find("div", id="main-content")))[0][4:].strip()      
                
                if len(title) == 0:
                    title = info[2]     

            except IndexError:
                title = info[2]

            if ("小米" in title.lower()) or ("redmi" in title.lower()) or ("poco" in title.lower()):
                title = "Xiaomi" + " " + title
            if ("紅米" in title.lower()):
                title = "Redmi" + " " + title
            if ("三星" in title.lower()):
                title = "Samsung" + " " + title

            title = " ".join(re.findall(r"[a-zA-Z0-9+]+", title))
            storage = None
            if "rog" in title.lower():
                for phone in phone_dict["Asus"]:
                    if phone.lower() in title.lower():
                        for s in  ["32", "64", "128", "256", "512"]:
                            if s in title:
                                storage = int(s)
                                break
                        title = "Asus" + " " + phone
                        break

            samsung_matches = ["note20ultra", "s20ultra", "s21ultra"]
            if any(x in title.lower().replace(" ", "") for x in samsung_matches):
                for phone in phone_dict["Samsung"]:
                    if phone.lower().replace(" ", "") in title.lower().replace(" ", ""):
                        for s in  ["32", "64", "128", "256", "512"]:
                            if s in title:
                                    storage = int(s)
                                    break
                        title = "Samsung Galaxy" + " " + phone
                        break

            for k in phone_dict.keys():
                if k.lower() in title.lower().replace(" ", ""): 
                    for phone in phone_dict[k]:
                        if phone.lower().replace(" ", "") in title.lower().replace(" ", ""):
                            for s in  ["32", "64", "128", "256", "512"]:
                                if s in title:
                                    storage = int(s)
                                    break
                            if k == "Samsung":
                                title = k + " " + "Galaxy" + " " + phone
                            elif k == "iPhone" or k == "Pixel":
                                title = phone
                            else:
                                title = k + " " + phone
                            break
                    break

            if title.split(" ")[0] in phone_dict.keys():
                pass
            else:
                for x in iphone_matches:
                    if x.lower().replace(" ", "") in title.lower().replace(" ", ""):
                        print(x)
                        for phone in phone_dict["iPhone"]:
                            if x == phone.lower().replace(" ", ""):
                                for s in  ["32", "64", "128", "256", "512"]:
                                    if s in title:
                                        storage = int(s)
                                        break
                                title = phone
                                print("iPhone:", title)
                                break
                        break

            try:
                account = info[0]
                created_at = datetime.strptime(info[3][3:].strip(), '%b %d %H:%M:%S %Y')
            except IndexError:
                account = None
                created_at = " ".join(re.findall(r'時間.+\n', str(soup.find("div", id="main-content")))[0].split(" ")[2:]).strip()
                created_at = datetime.strptime(created_at, '%b %d %H:%M:%S %Y')


            # status
            try:

                if ("全新" in info[2]):
                    new = True
                elif ("近全新" in info[2]) or ("外觀全新" in info[2]) or ("全新耳機" in info[2]):
                    new = False
                else:
                    new = False
            except IndexError:
                new = False

            # price
            try:
                wording = "價格"
                price = re.findall(r'售價格.+\n', str(soup.find("div", id="main-content")))
                if len(price) == 0:
                    wording = "金額"
                    price = re.findall(r'售金額.+\n', str(soup.find("div", id="main-content")))
                price = price[0].strip().replace(",", "")
                price = int(re.search(r'\d+', price).group())
            except IndexError:
                if wording == "金額":
                    price = re.findall(r'售金額.+', str(soup.find("div", id="main-content")).replace("\n", "").replace(",", "").replace(" ", ""))
                    price = int(re.findall(r'\d+', price)[0])
                else:
                    price = re.findall(r'售價格.+', str(soup.find("div", id="main-content")).replace("\n", "").replace(",", "").replace(" ", ""))[0].strip()
                    price = int(re.findall(r'\d+', price)[0])
            except AttributeError:
                sold = True
                price = None
            
            # items included
            try:
                box = re.findall(r'盒裝.+\n', str(soup.find("div", id="main-content")))[0].strip()
                if len(box) <= 5:
                    box = re.findall(r'盒裝內含.+購買', str(soup.find("div", id="main-content")).replace("\n", ""))[0]
                    box = box.replace(" ", "")[:-2]
            except IndexError:
                box = None

            data.append([title, storage, price, new, sold, account, box, url, created_at])
            raw_data.append([url, str(soup.find("body"))])
            db.ping()
            cursor = db.cursor(pymysql.cursors.DictCursor)  
            cursor.execute('''
                INSERT INTO ptt
                (`title`, `storage`, `price`, `new`, `sold`, 
                    `account`, `box`, `link`, `created_at`, 
                    `source`)
                VALUES (%s, %s, %s, %s, %s,
                        %s, %s, %s, %s, %s,
                        %s)
                ON DUPLICATE KEY UPDATE `title` = %s, 
                                        `storage` = %s, 
                                        `price` = %s, 
                                        `new` = %s, 
                                        `sold` = %s, 
                                        `account` = %s, 
                                        `box` = %s, 
                                        `created_at` = %s, 
                                        `source` = %s;
                ''', (title, storage, price, new, sold, account, box, url, created_at, "mobilesales",
                    title, storage, price, new, sold, account, box, created_at, "mobilesales"))
            db.commit()
            print(url)

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

USER = os.getenv("SQL_USER")
PWD = os.getenv("SQL_PSW")
HOST = os.getenv("SQL_HOST")

import pymysql
db = pymysql.connect(host=HOST, user=USER, password=PWD, database="smartphone")
cursor = db.cursor(pymysql.cursors.DictCursor)
cursor.execute('''
                SELECT * FROM ptt
                WHERE title = %s
               ''', "Pixel 2")
result = cursor.fetchall()
db.close()

In [ ]:
links = [ d["link"] for d in result]
links

In [63]:
raw_data = []
data = []
error_links = []
crawl_mobilesales(links, phone_dict, False)

NameError: name 'page' is not defined